# Phishing website detection using LSTM ## Problem 4: Language model

In [1]:
'''
author: Vinayak Kumar, email:vinayaksharan.er@gmail.com, dataset taken from:https://research.aalto.fi/en/datasets/phishstorm--phishing--legitimate-url-dataset(f49465b2-c68a-4182-9171-075f0ed797d5).html
'''

import pandas as pd
from string import printable
from sklearn.cross_validation import train_test_split
df1=pd.read_csv("urlset.csv",encoding='latin-1')

C:\Users\40750\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
domain=list(df1['domain'])
labels=list(df1['label'])
for i in domain:
    if i.startswith('\'') and i.endswith('\''):
        indx=domain.index(i)
        i=i.replace('\'','')
        domain[indx]=i 

In [3]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
#from keras.initializers import glorot_uniform
from keras.preprocessing.text import Tokenizer
from keras import regularizers

C:\Users\40750\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
max_len = 75
url_int_tokens = [[printable.index(x) + 1 for x in d if x in printable] for d in domain]
X = sequence.pad_sequences(url_int_tokens, maxlen=max_len)
print(type(X))
X_train, X_test, y_train, y_test=train_test_split(X,labels,test_size=0.25,random_state=42)

<class 'numpy.ndarray'>


In [5]:
def phising():
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')
    emb = Embedding(input_dim=110, output_dim=32, input_length=max_len,
                        dropout=0.2, W_regularizer=regularizers.l2(1e-4))(main_input)
    X = LSTM(32)(emb)
    X =  Dropout(0.5)(X)
    X = Dense(1, activation='sigmoid', name='output')(X)
    model = Model(inputs=[main_input], outputs=X)
    return model

In [6]:
model=phising()

C:\Users\40750\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  after removing the cwd from sys.path.
C:\Users\40750\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(input_dim=110, output_dim=32, input_length=75, embeddings_regularizer=<keras.reg...)`
  after removing the cwd from sys.path.


In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, epochs=15, batch_size=1000, verbose=1)

Epoch 1/15
71931/71931 [==============================] - 29s 404us/step - loss: 0.5706 - acc: 0.7094
Epoch 2/15
71931/71931 [==============================] - 28s 395us/step - loss: 0.3945 - acc: 0.8254
Epoch 3/15
71931/71931 [==============================] - 29s 403us/step - loss: 0.3485 - acc: 0.8505
Epoch 4/15
71931/71931 [==============================] - 29s 398us/step - loss: 0.3295 - acc: 0.8594
Epoch 5/15
71931/71931 [==============================] - 30s 417us/step - loss: 0.3122 - acc: 0.8678
Epoch 6/15
71931/71931 [==============================] - 30s 416us/step - loss: 0.3006 - acc: 0.8753
Epoch 7/15
71931/71931 [==============================] - 32s 446us/step - loss: 0.2878 - acc: 0.8814
Epoch 8/15
71931/71931 [==============================] - 35s 482us/step - loss: 0.2768 - acc: 0.8878
Epoch 9/15
71931/71931 [==============================] - 35s 489us/step - loss: 0.2662 - acc: 0.8931
Epoch 10/15
71931/71931 [==============================] - 31s 428us/step - loss: 

In [9]:
model.evaluate(X_test, y_test, verbose=1)

23977/23977 [==============================] - 8s 344us/step


[0.20806974786114552, 0.9180464611944615]

In [10]:
def predict(x_input):
    #print(x_input)
    url_int_tokens_1=[[printable.index(x) + 1 for x in x_input if x in printable]]
    #print(url_int_tokens_1)
    X_1 = sequence.pad_sequences(url_int_tokens_1, maxlen=max_len)
    p = model.predict(X_1)
    print(p)
    #print(model.predict(X_1))
    return "not_malicious" if p < 0.6 else "malicious"
x_input="https://lnkd.in/dkk5-Ti"
print(predict(x_input))

[[0.9139967]]
malicious
